In [2]:
import torch.nn.functional as F

import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from pandas import read_excel, read_json
import sys
import time

/home/remi/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())

[[92.72808837890625, 75.96248626708984], [74.3805160522461, 90.60895538330078]]


In [9]:
print(embeddings.size())

torch.Size([4, 384])


In [13]:
scores = (embeddings @ embeddings.T) * 100
print(scores.tolist())

[[100.0000228881836, 78.21473693847656, 92.72808837890625, 75.96248626708984], [78.21473693847656, 99.99999237060547, 74.3805160522461, 90.60895538330078], [92.72808837890625, 74.3805160522461, 100.00001525878906, 71.91613006591797], [75.96248626708984, 90.60895538330078, 71.91613006591797, 100.00001525878906]]


In [3]:
queries = read_excel("/home/remi/Downloads/Export.recherches.site.du.01-01-23.au.31-08-23.xlsx")

In [4]:
queries

,Nom,Recherches
0,procédure de licenciement salarié à domicile,23751
1,contrat de travail PARTICULIER EMPLOYEUR,10884
2,je veux démissionner,10537
3,congés payés,5838
4,3239,5591
...,...,...
495,paiement solde de tout compte,33
496,R4141-4,33
497,1090,32
498,3127,32


In [34]:
export_recherches = queries["Nom"].apply(lambda q : "query:"+str(q)).to_list()

In [35]:
len(export_recherches)

500

In [47]:
batch_dict = tokenizer(export_recherches, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:250] @ embeddings[250:].T) * 100
print(scores.tolist())

[[79.07263946533203, 95.72749328613281, 79.95138549804688, 86.67266082763672, 82.55464935302734, 90.29847717285156, 83.69933319091797, 81.92071533203125, 83.926513671875, 84.10591888427734, 86.33470153808594, 80.8721923828125, 83.51313018798828, 82.04386138916016, 83.52519226074219, 81.36088562011719, 84.707763671875, 83.32687377929688, 83.6168441772461, 79.84937286376953, 81.88897705078125, 83.05801391601562, 75.44232940673828, 86.98026275634766, 79.40036010742188, 81.496337890625, 85.5594253540039, 84.65572357177734, 87.9485092163086, 83.86797332763672, 82.49827575683594, 79.95145416259766, 84.61571502685547, 79.22595977783203, 84.00694274902344, 77.5650863647461, 85.58197021484375, 82.1629638671875, 82.87891387939453, 84.46195983886719, 79.16678619384766, 85.03498077392578, 83.05028533935547, 85.1852798461914, 84.44428253173828, 82.8910140991211, 82.08726501464844, 82.5843734741211, 81.58612060546875, 81.36625671386719, 77.45143127441406, 84.81259155273438, 84.23539733886719, 85.088

In [57]:
torch.max(scores[0])

tensor(95.7275, grad_fn=<MaxBackward1>)

In [105]:
for i in range(0, 250):    
    j = torch.argmax(scores[i]).item()
    matches = torch.topk(scores[i], 3).values.tolist()
    matching_indices= torch.topk(scores[i], 3).indices.tolist()
    print(matches)
    
    print(export_recherches[i])
    for m in matching_indices :
        print(export_recherches[250+m])
    print()

[95.72749328613281, 90.29847717285156, 89.406982421875]
query:procédure de licenciement salarié à domicile
query:procédure de licenciement
query:entretien préalable au licenciement
query:préavis licenciement

[91.88841247558594, 90.9750747680664, 90.3624038696289]
query:contrat de travail PARTICULIER EMPLOYEUR
query:Contrat de travail
query:particulier employeur préavis
query:activité partielle et réduction du temps de travail

[91.331787109375, 90.93653869628906, 90.11116790771484]
query:je veux démissionner
query:préavis de démission
query:arrêt de travail et démission
query:Abandon de poste

[96.18926239013672, 95.69268035888672, 95.5316162109375]
query:congés payés
query:période des congés payés
query:indemnités congé payés
query:Congé payé

[87.40385437011719, 85.40725708007812, 85.10961151123047]
query:3239
query:3127
query:2216
query:1486

[92.54462432861328, 91.8731460571289, 91.34576416015625]
query:rupture conventionnelle cdi
query:indemnités rupture conventionnelle
query:rup

In [94]:
torch.topk(scores[0], 3).values.tolist()

[95.72749328613281, 90.29847717285156, 89.406982421875]

In [71]:
export_recherches[154]

'query:panier repas'

In [4]:
fiches_mt = read_json("/home/remi/Downloads/fiches-min-travail.json")
passages = fiches_mt["text_full"].apply(lambda q : "query:"+str(q)).to_list()

In [7]:
fiches_mt["text_full"]

0      1) Visite d’information et de prévention : pou...
1      Question 1 : Jusqu’à quel âge ? Pour quelle qu...
2      Question 1 : Comment faire reconnaître son han...
3      1)\tComment est fixé le salaire minimum ? \n\n...
4      Question 1 VAE : à quoi ça sert et pour quelle...
                             ...                        
292    Les risques liés au chargement et au déchargem...
293    Qu’est-ce que le stockage des produits chimiqu...
294    L’obligation de substitution. De quoi s’agit-i...
295    A la suite d’un accident de travail mortel sur...
296    La réglementation française fixe un certain no...
Name: text_full, Length: 297, dtype: object

In [3]:
questions_raw = read_excel("/home/remi/Downloads/questions-fulltext.xlsx", header=None).dropna()

In [4]:
questions = questions_raw[0].apply(lambda q : "query:"+str(q)).to_list()

In [11]:
full_data = passages + questions

In [ ]:
# embbed each individually
p_emb = []
i = 0
for p in passages[:50] :
    i+=1
    print(i)
    #print(sys.getsizeof(p_emb))
    batch_dict = tokenizer(p, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    p_emb.append(F.normalize(embeddings, p=2, dim=1))
    
    time.sleep(1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


In [11]:
p_emb[0].size()

torch.Size([1, 384])

In [27]:
p_emb[3].element_size() * p_emb[3].nelement()

NameError: name 'p_emb' is not defined

In [25]:
1536*50

76800

In [6]:
embeddings

tensor([[ 1.9066e-02, -6.5092e-03, -1.7773e-01, -2.6187e-01,  2.2347e-01,
         -2.3642e-01,  2.2102e-01,  4.4021e-01,  1.5216e-01,  2.9175e-02,
          2.3398e-01,  1.0883e-01,  3.3503e-01, -1.2731e-01,  4.1160e-02,
          1.6390e-01,  4.4923e-01, -1.6416e-01, -1.0002e-01, -2.6164e-01,
          5.0304e-02, -1.5459e-01, -1.9049e-01,  1.9551e-01,  2.9928e-01,
          2.3421e-01, -3.4678e-05,  3.6246e-01,  1.5929e-01, -2.3409e-01,
         -1.5682e-01, -3.4468e-02,  1.7295e-01, -3.6459e-01,  5.0336e-02,
          7.4437e-03, -2.5104e-01, -2.5032e-01,  2.7756e-01, -2.5578e-01,
         -3.9132e-01,  2.2648e-01,  2.3583e-02,  4.7398e-01,  1.4859e-01,
          2.2563e-01, -1.6374e-01,  1.6732e-01, -1.3053e-02, -2.6316e-01,
         -1.6557e-01,  1.9343e-01, -1.0065e-01,  2.1225e-01, -2.3312e-02,
         -3.8587e-01, -1.1679e-01, -1.8573e-01, -3.4832e-01,  2.8454e-01,
          2.4632e-01,  8.1678e-02,  1.5831e-01,  6.8732e-02,  2.0884e-01,
          4.6081e-02,  3.0048e-01,  2.

tensor([[ 4.1098e-03, -1.4031e-03, -3.8310e-02, -5.6447e-02,  4.8170e-02,
         -5.0961e-02,  4.7640e-02,  9.4887e-02,  3.2798e-02,  6.2887e-03,
          5.0435e-02,  2.3459e-02,  7.2216e-02, -2.7442e-02,  8.8722e-03,
          3.5328e-02,  9.6832e-02, -3.5385e-02, -2.1559e-02, -5.6398e-02,
          1.0843e-02, -3.3323e-02, -4.1061e-02,  4.2143e-02,  6.4510e-02,
          5.0485e-02, -7.4749e-06,  7.8128e-02,  3.4335e-02, -5.0458e-02,
         -3.3802e-02, -7.4296e-03,  3.7279e-02, -7.8589e-02,  1.0850e-02,
          1.6045e-03, -5.4112e-02, -5.3957e-02,  5.9830e-02, -5.5134e-02,
         -8.4350e-02,  4.8818e-02,  5.0834e-03,  1.0217e-01,  3.2029e-02,
          4.8635e-02, -3.5295e-02,  3.6066e-02, -2.8137e-03, -5.6725e-02,
         -3.5688e-02,  4.1693e-02, -2.1696e-02,  4.5750e-02, -5.0250e-03,
         -8.3175e-02, -2.5174e-02, -4.0035e-02, -7.5080e-02,  6.1333e-02,
          5.3095e-02,  1.7606e-02,  3.4123e-02,  1.4815e-02,  4.5016e-02,
          9.9329e-03,  6.4770e-02,  5.